In [39]:
##Functions and libraries
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from torchsummary import summary
import torch.nn as nn
import evaluate
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine
from fpdf import FPDF
from sklearn.metrics import f1_score

##Point to the local server and the model to be used and set up the stable prompt contents
client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")
model = SentenceTransformer("all-MiniLM-L6-v2")

system_prompt =  "Rate how positive or a negative a clothing article review is. Only answer with a single digit from 1 to 5, use '1' for negative, '2' for somewhat negative, '3' for neutral, '4' for somewhat positive and '5' for positive. Do not answer with anything else than '1' or '2' or '3' or '4' or '5'."
system_prompt_RAG = "Rate how positive or a negative a clothing article review is. Only answer with a single digit from 1 to 5, use '1' for negative, '2' for somewhat negative, '3' for neutral, '4' for somewhat positive and '5' for positive. Do not answer with anything else than '1' or '2' or '3' or '4' or '5'. Use the ratings of the following similar reviews to make your decision. "
user_prompt_stable =  "Rate how positive or a negative a clothing article review is. Only answer with a single digit from 1 to 5, use '1' for negative, '2' for somewhat negative, '3' for neutral, '4' for somewhat positive and '5' for positive. Do not answer with anything else than '1' or '2' or '3' or '4' or '5'. The review goes as follows: "

##Dataset
# Decode the review froms the dataset, only relevant for IMDB
# def decode_reviews(sequences):
    
#     return [
#         " ".join([reverse_word_index.get(i - 3, "?") for i in sequence[1:]])
#         for sequence in sequences
#     ]


##Normal Prompting procedure
# API access to the model
def prompt_model(prompt):
    completion = client.chat.completions.create(
        model="llama-3-8b-lexi-uncensored",
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {"role": "user", "content": user_prompt_stable + str(prompt)},
        ],
        temperature=0,
    )
    return completion.choices[0].message.content


##RAG Magic
# Create vector embeddings for the reviews
def encode_text_list(text_list):
    embeddings = [model.encode(text) for text in text_list]
    return embeddings


# return the indices of the top n relevant reviews for retrieval
def calculate_top_n_similarities(prompt, stored_embeddings, top_n=3):
    # Encode the new prompt
    prompt_embedding = model.encode(prompt)

    # Calculate cosine similarity with each stored embedding
    similarities = [
        1 - cosine(prompt_embedding, emb) for emb in stored_embeddings
    ]  # 1 - cosine_distance = cosine_similarity. The higher the value, the more similar the vectors are

    # Get the indices of the top n similarities
    top_n_indices = np.argsort(similarities)[-top_n:][::-1]

    return top_n_indices


# construct context query for the model
def integrate_knowledge(stored_reviews, top_n_indices):
    # Define ordinal words for readability
    ordinals = [
        "The most similar",
        "The second most similar",
        "The third most similar",
        "The fourth most similar",
        "The fifth most similar",
        "The sixth most similar",
        "The seventh most similar",
        "The eighth most similar",
        "The ninth most similar",
        "The tenth most similar",
    ]
    # Construct the formatted string by integrating the reviews in the correct order
    integrated_string = system_prompt_RAG
    for i, idx in enumerate(top_n_indices):
        # Get the ordinal word based on index position, or use "next" if ordinals are exceeded
        ordinal_word = ordinals[i] if i < len(ordinals) else "next"
        # Append the formatted review to the integrated string
        integrated_string += f"{ordinal_word} review, {stored_reviews[idx]}"

    return integrated_string.strip()


# RAG-prompts for the model
def RAG_prompt_model(prompt, n_known_reviews=5):
    top_n_indices = calculate_top_n_similarities(prompt, stored_embeddings, top_n=n_known_reviews)
    integragted_prompt = integrate_knowledge(review_list, top_n_indices)
    completion = client.chat.completions.create(
        model="llama-3-8b-lexi-uncensored",
        messages=[
            {
                "role": "system",
                "content": integragted_prompt,
            },
            {"role": "user", 
             "content": user_prompt_stable+ str(prompt)},
        ],
        temperature=0,
    )
    return completion.choices[0].message.content


# Generate RAG document for AnythingLLM
# def write_reviews_to_txt(reviews, ratings, output_filename="movie_reviews.txt"):
#     # Check that both lists have the same length
#     if len(reviews) != len(ratings):
#         raise ValueError("The number of reviews and ratings must be the same.")

#     try:
#         with open(output_filename, "w", encoding="utf-8") as file:
#             # Iterate through reviews and ratings
#             for review, rating in zip(reviews, ratings):
#                 # Format the review text
#                 formatted_review = f'The review: "{review}" was rated "{rating}".\n'

#                 # Write the formatted review to the file
#                 file.write(formatted_review)
#                 file.write("\n")  # Add a blank line between reviews

#         print(f"Text file '{output_filename}' has been created successfully.")
#     except Exception as e:
#         print(f"Error saving text file: {e}")


# generate list of reviews and ratings
def store_reviews_with_ratings(reviews, ratings):
    # Check that both lists have the same length
    if len(reviews) != len(ratings):
        raise ValueError("The number of text inputs and ratings must be the same.")

    # List to store the formatted reviews with ratings
    formatted_reviews = []

    # Iterate through reviews and ratings
    for review, rating in zip(reviews, ratings):
        # Format the review text
        formatted_review = f'the review: "{review}" was rated "{rating}".'

        # Append the formatted review to the list
        formatted_reviews.append(formatted_review)

    return formatted_reviews


# Working in bigger batches with progress feedback
def batch_prompt_model(prompts, mode="normal"):
    results = []
    for p in prompts:
        if mode == "normal":
            results.append(prompt_model(p))
        elif mode == "RAG":
            results.append(RAG_prompt_model(p))
        else:
            raise ValueError(f"Invalid mode '{mode}'.")
        if len(results) % 10 == 0:
            print(f"Completed {len(results)} prompts.")
    return results


##Handling unusual outputs
# Helper funtion to handle weird outputs
def convert_outputs(strings):
    result = []
    weird = 0
    skipped = 0
    skipped_indices = []

    for i, s in enumerate(strings):
        if s not in ["1", "2", "3", "4", "5"]:
            weird += 1
        # Filter out any characters that are not '1' or '0'
        cleaned = "".join([char for char in s if char in "12345"])

        # Convert to integer if the cleaned string is exactly "1" or "0"
        if cleaned == "1":
            result.append(1)
        elif cleaned == "2":
            result.append(2)
        elif cleaned == "3":
            result.append(3)
        elif cleaned == "4":
            result.append(4)
        elif cleaned == "5":
            result.append(5)
        else:
            # Handle unexpected cases if needed; here we skip them
            print(f"Warning: Unrecognized format '{s}', skipping.")
            print("")
            skipped += 1
            skipped_indices.append(i)
    if weird > 0:
        print(f"This batch query produced {weird} weird outputs.")
    if skipped > 0:
        print(
            f"Additionally, it skipped {skipped} outputs that did not contain 1 or 0 at all."
        )
    return result, skipped_indices


# delete the skipped indices from the list if needed
def clean_y(y, skipped_indices):
    if len(skipped_indices) > 0:
        print(f"Warning: removing {len(skipped_indices)} skipped outputs from y.")
        return [y[i] for i in range(len(y)) if i not in skipped_indices]
    else:
        return y

C:\Users\LaptopBraun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


We will test normal, non-augemented prompting first.
The dataset contains movie reviews either rated positive or negative

In [40]:
##Dataset
x_split = 5000
reviews_data = pd.read_csv("Clothing Reviews.csv").dropna(subset=["Review Text","Rating"]).sample(frac=1).reset_index(drop=True) #shuffle the dataset every time
train_x, train_y, test_x, test_y = reviews_data["Review Text"][:x_split].values, reviews_data["Rating"][:x_split].values, reviews_data["Review Text"][x_split:].values, reviews_data["Rating"][x_split:].values
#set_size = len(reviews_data)-x_split
set_size = 500
x = test_x[:set_size]
y = test_y[:set_size]

In [41]:
#Select a subset of the dataset and try the basic model
#cats, ids = convert_outputs(batch_prompt_model(x))
# valid_test_y = clean_y(y, ids)
# print(f"Weighted F1-Score of standard prompting: {f1_score(valid_test_y, cats, average='weighted')}")

No we will test the simple RAG implementation set-up in this script

In [42]:
#Generate RAG Knowledge-Base
# RAG_size = x_split #full size
# review_list = store_reviews_with_ratings(train_x[:RAG_size], train_y[:RAG_size])
# stored_embeddings = encode_text_list(review_list)

In [43]:
#cats, ids = convert_outputs(batch_prompt_model(x, mode="RAG"))
#valid_test_y = clean_y(test_y[:set_size], ids)
#print(f"Weighted F1-Score of RAG prompting: {f1_score(valid_test_y, cats, average='weighted')}")

Let's try to benchmark this performance against a transfer learning approach

In [44]:
#Define the model architecture class
class MulticlassTextClassificationModel(nn.Module):
    def __init__(self, model_name, num_labels):
        super(MulticlassTextClassificationModel, self).__init__()
        # Load the pre-trained model (e.g., BERT, RoBERTa, etc.)
        self.model = AutoModel.from_pretrained(model_name)
        
        # Add a larger dense layer (5000 neurons)
        self.dense_layer = nn.Linear(self.model.config.hidden_size, 32)
        self.relu = nn.ReLU()  # Activation function for the dense layer
        
        # Final classification layer
        self.classifier = nn.Linear(32, num_labels)  # This layer now takes 5000 neurons as input

    def forward(self, input_ids, attention_mask, labels=None):
        # Get model outputs
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use [CLS] token output for classification (outputs.last_hidden_state[:, 0, :])
        cls_output = outputs.last_hidden_state[:, 0, :]
        
        # Pass the CLS token output through the dense layer
        dense_output = self.dense_layer(cls_output)
        dense_output = self.relu(dense_output)  # Apply ReLU activation
        
        # Classifier layer to get the final logits
        logits = self.classifier(dense_output)
        
        loss = None
        if labels is not None:
            # Compute loss using CrossEntropyLoss
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return {"loss": loss, "logits": logits}

#Definfe the dataset structure class
class CustomDataset(Dataset):
    def __init__(self, tokenized_data, labels):
        self.input_ids = tokenized_data["input_ids"]
        self.attention_mask = tokenized_data["attention_mask"]
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if isinstance(idx, slice):
            # Return a new CustomDataset instance for a slice
            return CustomDataset(
                tokenized_data={
                    "input_ids": self.input_ids[idx],
                    "attention_mask": self.attention_mask[idx]
                },
                labels=self.labels[idx] -1 # Adjust labels to start from 0
            )
        else:
            # Return a single sample as usual
            return {
                "input_ids": self.input_ids[idx],
                "attention_mask": self.attention_mask[idx],
                "labels": self.labels[idx] -1 # Adjust labels to start from 0
            }
    def __str__(self):
        # Display a summary of the dataset
        return (
            f"CustomDataset with {len(self)} samples\n"
            f"Input IDs: {self.input_ids.shape}\n"
            f"Attention Masks: {self.attention_mask.shape}\n"
            f"Labels: {self.labels.shape}"
        )

    def __repr__(self):
        # Display more details about the dataset for debugging
        content_preview = {
            "input_ids": self.input_ids[:3].tolist(),
            "attention_mask": self.attention_mask[:3].tolist(),
            "labels": self.labels[:3].tolist(),
        }
        return (
            f"CustomDataset(length={len(self)})\n"
            f"Preview of data:\n{content_preview}"
        )
        
# Load the F1 metric from the evaluate library
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, label_ids = eval_pred  # Unpack EvalPrediction object
    
    # Convert logits to predicted class labels
    preds = np.argmax(predictions, axis=1)
    
    # Compute weighted F1 score
    results = f1_metric.compute(predictions=preds, references=label_ids, average="weighted")
    
    return results
#Dataset prepraration
eval_set_size = len(train_x) // 20  # 20% of the training set for evaluation
max_length = 200 #max text token length
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the text column for the train set
tokenized_train = tokenizer(
    train_x[eval_set_size:].tolist(),
    truncation=True,
    padding='max_length',
    max_length=max_length,
    return_tensors="pt"  # Directly return PyTorch tensors
)
# Add the labels as tensors
train_labels = torch.tensor(train_y[eval_set_size:])
train_dataset = CustomDataset(tokenized_train, train_labels)


# Tokenize the text column for the eval set
tokenized_eval = tokenizer(
    test_x[:eval_set_size].tolist(),
    truncation=True,
    padding='max_length',
    max_length=max_length,
    return_tensors="pt"  # Directly return PyTorch tensors
)
# Add the labels as tensors
eval_labels = torch.tensor(train_y[:eval_set_size])
eval_dataset = CustomDataset(tokenized_eval, eval_labels)
#Define the model and training arguments
model = MulticlassTextClassificationModel(model_name, num_labels=5)

training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs=3, per_device_train_batch_size=380, evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)
# Freeze BERT layers (set requires_grad=False for all parameters in the BERT model)
for param in model.model.parameters():
    param.requires_grad = False

# Unfreeze the final custom layers (dense layer and classifier)
for param in model.dense_layer.parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True

# Check if BERT layers are frozen
for name, param in model.model.named_parameters():
    if param.requires_grad:
        print(f"Trainable BERT layer: {name}")
    else:
        print(f"Frozen BERT layer: {name}")

# Check if custom layers are trainable
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable layer: {name} with {param.numel()} parameters")
    else:
        print(f"Frozen layer: {name} with {param.numel()} parameters")


C:\Users\LaptopBraun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Frozen BERT layer: embeddings.word_embeddings.weight
Frozen BERT layer: embeddings.position_embeddings.weight
Frozen BERT layer: embeddings.LayerNorm.weight
Frozen BERT layer: embeddings.LayerNorm.bias
Frozen BERT layer: transformer.layer.0.attention.q_lin.weight
Frozen BERT layer: transformer.layer.0.attention.q_lin.bias
Frozen BERT layer: transformer.layer.0.attention.k_lin.weight
Frozen BERT layer: transformer.layer.0.attention.k_lin.bias
Frozen BERT layer: transformer.layer.0.attention.v_lin.weight
Frozen BERT layer: transformer.layer.0.attention.v_lin.bias
Frozen BERT layer: transformer.layer.0.attention.out_lin.weight
Frozen BERT layer: transformer.layer.0.attention.out_lin.bias
Frozen BERT layer: transformer.layer.0.sa_layer_norm.weight
Frozen BERT layer: transformer.layer.0.sa_layer_norm.bias
Frozen BERT layer: transformer.layer.0.ffn.lin1.weight
Frozen BERT layer: transformer.layer.0.ffn.lin1.bias
Frozen BERT layer: transformer.layer.0.ffn.lin2.weight
Frozen BERT layer: transf

C:\Users\LaptopBraun\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

  0%|          | 0/39 [00:00<?, ?it/s]